In [ ]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr

### Create cloud mask image

You can create cloud mask image with using SCL band. <br>
scl_list >> default= [3,8,9,10] <br>
SCL Bands List:0 - No data
> 1 - Saturated / Defective <br>
>2 - Dark Area Pixels <br>
>3 - Cloud Shadows <br>
>4 - Vegetation <br> 
>5 - Bare Soils <br>
>6 - Water <br>
>7 - Clouds low probability / Unclassified <br>
>8 - Clouds medium probability <br> 
>9 - Clouds high probability <br>
>10 - Cirrus <br>
>11 - Snow / Ice <br>


In [ ]:
target_aoi={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}

datajson=json.dumps(target_aoi)
area=gpd.read_file(datajson)

#cpy.VectorProcessing.show_vector(target_area=area)
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')
#m
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-08-01/2018-12-01'
cloud_percentage=30
band_list=['B02','B08']

In [ ]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)


In [ ]:
t_list=stac_query.create_tiles_list()
t_list

In [ ]:
stac_query.tiles_list=['37SDA']
stac_query.find_time_series()


In [ ]:
%%time
dow=cpy.Stac.download_subset_image(item_list=stac_query.stac_items[0:1],
                                   aoi=target_aoi,
                                   band_list=['B09','B04'],
                                   download_status=True,
                                   ###### OPEN CLOUD MASKING ################
                                   cloud_masking=True,
                                   )

In [ ]:
dow[0]['B04']

#### FIND BEST IMAGES

In [ ]:
sentinel_items=stac_query.find_sentinel_item()

In [ ]:
%%time
dow=cpy.Stac.download_subset_image(item_list=sentinel_items,
                                   aoi=target_aoi,
                                   band_list=['B09','B04'],
                                   #download_status=True,
                                   ###### OPEN CLOUD MASKING ################
                                   cloud_masking=True,
                                   )